<a href="https://colab.research.google.com/github/Nick088Official/Gemma-Google-Colab/blob/main/Gemma_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run Google Gemma AI Models
Check out the models: https://blog.google/technology/developers/gemma-open-models/

Made by [Nick088](https://linktr.ee/Nick088)

In [ ]:
#@title Install & Load Dependencies, Model
!pip install einops
!pip install accelerate
!pip install huggingface_hub
import torch
from huggingface_hub import login
from IPython.display import clear_output
from huggingface_hub import hf_hub_download

if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

Google_Gemma_Model = "gemma-7b" #@param ['gemma-2b', 'gemma-2b-it', 'gemma-7b', 'gemma-7b-it']

#@markdown The normal version are the official ones, the other ones are GGUF quantized, compressed to consume less ram.

GGUF_Format = False #@param {type:"boolean"}

#@markdown Go to https://huggingface.co/settings/tokens and make a token with the read role, and paste it here.
Hugging_Face_Read_Token = "" #@param {type:"string"}

# Authenticate
login(token=Hugging_Face_Read_Token)

if GGUF_Format == False:
  from transformers import AutoModelForCausalLM, AutoTokenizer
  if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(f"google/{Google_Gemma_Model}", torch_dtype="auto", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(f"google/{Google_Gemma_Model}", trust_remote_code=True)
  else:
    model = AutoModelForCausalLM.from_pretrained(f"google/{Google_Gemma_Model}", torch_dtype=torch.float32, device_map="cpu", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(f"google/{Google_Gemma_Model}", trust_remote_code=True)
else:
  !nvidia-smi
  if device == "cuda":
    !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
  else:
    !pip install llama-cpp-python
  from huggingface_hub import hf_hub_download
  from llama_cpp import Llama
  model_name = f"google/{Google_Gemma_Model}"
  model_file = f"{Google_Gemma_Model}.gguf"
  model_path = hf_hub_download(model_name,
                             filename=model_file,
                             local_dir='/content',
                             token=Hugging_Face_Read_Token)
  from llama_cpp import Llama
  llm = Llama(model_path=model_path,
            n_gpu_layers=-1)

clear_output()
if GGUF_Format == False:
  print(f"Downloaded {Google_Gemma_Model}")
else:
  print(f"Downloaded {Google_Gemma_Model} in GGUF Format")

In [ ]:
#@title Run Gemma UI

def answer(user_prompt, system_prompt, max_new_tokens, repetition_penalty):
  if phi_2_model_version == "normal":
    # Concatenate system and user prompts
    full_prompt = system_prompt + user_prompt

    # Tokenize the combined prompt
    inputs = tokenizer(full_prompt, return_tensors="pt", return_attention_mask=False)

    # Generate text based on the combined prompt and additional inputs
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        repetition_penalty=repetition_penalty
    )

    # Extract the generated text from the model output
    text = tokenizer.batch_decode(outputs)[0]
    text = text[len(system_prompt):].strip()
    return text
  else:
    response = llm(full_prompt, max_tokens=max_new_tokens, repeat_penalty=repetition_penalty)
    response = response['choices'][0]['text']
    return response

# Define the gradio interface
interface = gr.Interface(
    fn=answer,
    inputs=[
        gr.Textbox(lines=2, label="Your Prompt"),
        gr.Textbox(lines=2, label="System Prompt"),
        gr.Slider(value=3000, minimum=256, maximum=8192, step=1, interactive=True, label="Max Tokens"),
        gr.Slider(value=1.2, minimum=0.0, maximum=2.0, step=0.1, interactive=True, label="Repetition Penalty"),
    ],
    outputs="text",
    title=f"{Google_Gemma_Model}",
)


# Launch the gradio interface
interface.launch(share=True, debug=True)